In [ ]:
import zipfile

!unzip /content/cats.zip

Archive:  /content/cats.zip
replace cats/27.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: cats/27.jpg             
  inflating: cats/28.jpg             
  inflating: cats/29.jpg             
  inflating: cats/30.jpg             
  inflating: cats/31.jpg             
  inflating: cats/32.jpg             


In [ ]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# Check device
device = th.device('cuda' if th.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 10  # Increase the number of epochs
batch_size = 32  # Increase the batch size
learning_rate = 0.001
num_classes = 2
classes = ('cats', 'dogs')

# Data transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(32),  # Use random resized crop instead of resize
    transforms.RandomHorizontalFlip(),  # Add random horizontal flip for data augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load train and test datasets
train_dataset = torchvision.datasets.ImageFolder('/content/train', transform=transform)
train_loader = th.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = torchvision.datasets.ImageFolder('/content/test', transform=transform)
test_loader = th.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the CNN architecture
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 220)
        self.fc2 = nn.Linear(220, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create the model instance and move it to the device
model = ConvNet().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model.parameters(), lr=learning_rate)  # Use Adam optimizer

# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()  # Set model to training mode

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()  # Set model to evaluation mode

with th.no_grad():
    correct = 0
    total = 0
    class_correct = [0 for _ in range(num_classes)]
    class_total = [0 for _ in range(num_classes)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs= model(images)
        _, predicted = th.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        for i in range(len(labels)):
            label = labels[i].item()
            pred = predicted[i].item()
            class_correct[label] += int(label == pred)
            class_total[label] += 1

    accuracy = 100 * correct / total
    print(f'Overall Accuracy: {accuracy:.2f}%')

    for i in range(num_classes):
        class_accuracy = 100 * class_correct[i] / class_total[i]
        print(f'Accuracy of {classes[i]}: {class_accuracy:.2f}%')


Overall Accuracy: 59.29%
Accuracy of cats: 42.86%
Accuracy of dogs: 75.71%


In [ ]:
from torchvision.io import read_image
paths = ('/content/cats/2016.jpg','/content/cats/27.jpg','/content/cats/28.jpg',
         '/content/cats/29.jpg','/content/cats/30.jpg','/content/cats/31.jpg',
         '/content/cats/32.jpg','/content/dog1.jpg')
for i in range(len(paths)):
  img = read_image(paths[i])
  img = transforms.ToPILImage()(img)
  p = transform(img)
  p.size()
  os = model(p)
  # max returns (value ,index)
  _, predicted = th.max(os, 1)
  predicted_label = classes[predicted]
  print(predicted_label)

dogs
dogs
cats
cats
cats
dogs
cats
dogs
